# Load data

In [7]:
from keras.datasets import mnist
from PIL import Image
import numpy as np
from keras.utils import to_categorical
from sklearn.utils import shuffle
import time
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras import optimizers
from keras import backend as K
import matplotlib.pyplot as plt

In [23]:
# --------------------- processing data ----------------------
# loading data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_classes = 10

# form one hot label
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

def resize_image(data):
    img = Image.fromarray(data.reshape(28,28))
    img = img.resize(size=(32,32))
    return np.array(img)

# preprocessing: resize to 32,32. scale to [0,1]
X_train = np.array([resize_image(x) for x in X_train])
X_test = np.array([resize_image(x) for x in X_test])




In [37]:
def rotation_imgs(X, degree):
    size = 32,32
    ans = []
    for x in X:
        img = Image.fromarray(x.reshape(size), mode='L')
        #print(img.mode)
        img.rotate(degree)
        ans.append(np.array(img))
        #print(np.array(img).shape)
    return np.array(ans)

from PIL import ImageFilter

def blur_imgs(X, radius):
    size = 32,32
    ans = []
    for x in X:
        img = Image.fromarray(x.reshape(size), mode='L')
        #print(img.mode)
        img.filter(ImageFilter.GaussianBlur(radius=radius))
        ans.append(np.array(img))
        #print(np.array(img).shape)
    return np.array(ans)
        
        
        

# Build NN

In [10]:
# ------------------- create model -------------------
start_time = time.time()
model = Sequential()
# add model layers
img_rows, img_cols = 32, 32
print(K.image_data_format())
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# 32, 32
channel_multiply = 4

model.add(Conv2D(1 * channel_multiply, (3, 3), activation='relu', padding='same',
                 input_shape=input_shape))
# 32, 32
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# 16, 16
model.add(Conv2D(2 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 16, 16
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# 8, 8
model.add(Conv2D(4 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 8, 8
model.add(Conv2D(4 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 8, 8
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# 4, 4
model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 4, 4
model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 4, 4
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# 2, 2
model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 2, 2
model.add(Conv2D(8 * channel_multiply, (3, 3), activation='relu', padding='same'))
# 2, 2
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
# 1, 1
model.add(Flatten())
model.add(Dense(4 * channel_multiply))
model.add(Dense(4 * channel_multiply))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

sgd = optimizers.SGD(lr=1e-8, decay=1e-6, momentum=0.9, nesterov=True)
adam = optimizers.Adamax(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-5)
ada = optimizers.Adadelta()

channels_last


In [38]:
# plot
def plot_img(Y1, Y2, title):
    plt.clf()
    plt.plot(Y1)
    plt.plot(Y2)
    plt.title(title)
    plt.ylabel(title)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(title + '.png')



In [12]:
X_train, y_train = shuffle(X_train, y_train, random_state=13)

In [44]:
def train(rotation=0, blur=0, epoches=5):
    n = X_train.shape[0]
    print(X_train.shape)
    print(X_test.shape)
    m = X_test.shape[0]
    X_train_ = blur_imgs(rotation_imgs(X_train, rotation),blur).reshape(n, 32,32,1).astype('float32')
    X_test_ = blur_imgs(rotation_imgs(X_test, rotation),blur).reshape(m, 32,32,1).astype('float32')
    
    X_train_ /= 255.0
    X_test_ /= 255.0

    model.compile(optimizer=adam, loss=keras.losses.categorical_crossentropy, 
                  metrics=['accuracy'])
    tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,
                                         write_graph=True, write_images=True)
    history = model.fit(X_train_, y_train, validation_data=(X_test_, y_test), 
                        epochs=epoches, batch_size=32, callbacks=[tbCallBack])
    plot_img(history.history['acc'], history.history['val_acc'], 
             "rotation_"+str(rotation) + ", blur_" + str(blur)+" Accuracy")
    plot_img(history.history['loss'], history.history['val_loss'], 
             "rotation_"+str(rotation) + ", blur_" + str(blur)+" Loss")



In [ ]:
for rotation in range(-45,45,5):
    train(rotation, 0, 5)
for blur in range(7):
    train(0, blur, 5)

(60000, 32, 32)
(10000, 32, 32)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 21s 351us/step - loss: 0.4815 - acc: 0.8470 - val_loss: 0.1594 - val_acc: 0.9503
Epoch 2/5
60000/60000 [==============================] - 21s 352us/step - loss: 0.1509 - acc: 0.9537 - val_loss: 0.0984 - val_acc: 0.9690
Epoch 3/5
60000/60000 [==============================] - 22s 364us/step - loss: 0.1042 - acc: 0.9682 - val_loss: 0.0767 - val_acc: 0.9751
Epoch 4/5
60000/60000 [==============================] - 21s 358us/step - loss: 0.0808 - acc: 0.9753 - val_loss: 0.0766 - val_acc: 0.9730
Epoch 5/5
60000/60000 [==============================] - 21s 352us/step - loss: 0.0673 - acc: 0.9790 - val_loss: 0.0594 - val_acc: 0.9811
(60000, 32, 32)
(10000, 32, 32)
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
54656/60000 [==========================>...] - ETA: 1s - loss: 0.0573 - acc: 0.9817

In [ ]:
# regularizer

In [ ]:

kernel_regularizer=regularizers.l2(0.01)

In [ ]:
# data augmentation